In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import fftpack
from scipy import signal
import numpy as np
import pandas as pd
import datetime
import time as T

In [2]:
#mozart

In [3]:
cd ~/Documents/SyncRecord/FFT-data/

/home/takeyama/Documents/SyncRecord/FFT-data


In [4]:
#my workstation

In [5]:
#cd /home/takeyama/Documents/01_SensorData/2016_05_11/FFT-data/

In [6]:
ls

FFT-Cleaning-Conv-left-hand-ags.csv-128bit.csv
FFT-Cleaning-Conv-left-hand-ags.csv-16bit.csv
FFT-Cleaning-Conv-left-hand-ags.csv-256bit.csv
FFT-Cleaning-Conv-left-hand-ags.csv-32bit.csv
FFT-Cleaning-Conv-left-hand-ags.csv-512bit.csv
FFT-Cleaning-Conv-left-hand-ags.csv-64bit.csv
FFT-Cleaning-Conv-left-leg-ags.csv-128bit.csv
FFT-Cleaning-Conv-left-leg-ags.csv-16bit.csv
FFT-Cleaning-Conv-left-leg-ags.csv-256bit.csv
FFT-Cleaning-Conv-left-leg-ags.csv-32bit.csv
FFT-Cleaning-Conv-left-leg-ags.csv-512bit.csv
FFT-Cleaning-Conv-left-leg-ags.csv-64bit.csv
FFT-Cleaning-Conv-right-hand-ags.csv-128bit.csv
FFT-Cleaning-Conv-right-hand-ags.csv-16bit.csv
FFT-Cleaning-Conv-right-hand-ags.csv-256bit.csv
FFT-Cleaning-Conv-right-hand-ags.csv-32bit.csv
FFT-Cleaning-Conv-right-hand-ags.csv-512bit.csv
FFT-Cleaning-Conv-right-hand-ags.csv-64bit.csv
FFT-Cleaning-Conv-right-leg-ags.csv-128bit.csv
FFT-Cleaning-Conv-right-leg-ags.csv-16bit.csv
FFT-Cleaning-Conv-right-leg-ags.csv-256bit.csv
FFT-Cleaning-Conv-right

In [7]:
fft_left_hand = pd.read_csv('FFT-'+'Cleaning-Conv-left-hand-ags.csv'+'-16'+'bit.csv')
#left_hand.Time=pd.to_timedelta(left_hand.Time)
fft_left_leg = pd.read_csv('FFT-'+'Cleaning-Conv-left-leg-ags.csv'+'-16'+'bit.csv')
#left_leg.Time=pd.to_timedelta(left_leg.Time)
fft_right_hand = pd.read_csv('FFT-'+'Cleaning-Conv-right-hand-ags.csv'+'-16'+'bit.csv')
#right_hand.Time=pd.to_timedelta(right_hand.Time)
fft_right_leg = pd.read_csv('FFT-'+'Cleaning-Conv-right-leg-ags.csv'+'-16'+'bit.csv')
#right_leg.Time=pd.to_timedelta(right_leg.Time)
fft_west = pd.read_csv('FFT-'+'Cleaning-Conv-west-ags.csv'+'-16'+'bit.csv')
#west.Time=pd.to_timedelta(west.Time)

In [8]:
fft_left_leg.dtypes

Time           int64
Number        object
AccX_imag    float64
AccX_real    float64
AccY_imag    float64
AccY_real    float64
AccZ_imag    float64
AccZ_real    float64
GyrX_imag    float64
GyrX_real    float64
GyrY_imag    float64
GyrY_real    float64
GyrZ_imag    float64
GyrZ_real    float64
dtype: object

In [9]:
#http://blog.vingow.com/tech/numpy-jensen-shannon/ よりKL距離の算出アルゴリズム
def kld(p, q):
    """Calculates Kullback–Leibler divergence"""
    p = np.array(p)
    q = np.array(q)
    return np.sum(p * np.log(p / q), axis=(p.ndim - 1))

In [10]:
x = fft_left_hand.where(fft_left_hand.Time==0).dropna()

In [11]:
x

,Time,Number,AccX_imag,AccX_real,AccY_imag,AccY_real,AccZ_imag,AccZ_real,GyrX_imag,GyrX_real,GyrY_imag,GyrY_real,GyrZ_imag,GyrZ_real
0,0.0,1st,-0.015192,0.358045,-0.869577,-1.991186,1.070767,0.360219,-32.797698,15.109767,-10.299680,5.271495,-5.125378,1.122696
1,0.0,2nd,-0.103970,-0.094514,0.583755,-0.466071,0.504561,-0.320706,-10.844850,3.459231,-3.601636,1.810484,-2.845803,1.582849
2,0.0,3rd,-0.104528,0.058390,0.365722,-0.080314,0.164233,-0.203757,-6.372311,3.403319,-2.309824,1.560055,-1.884029,1.299952
3,0.0,4th,-0.075054,-0.003471,0.100201,0.070954,0.156054,-0.184714,-4.014669,3.820903,-1.489992,1.605612,-1.263464,1.126835
4,0.0,5th,-0.044162,0.022334,0.024453,-0.080821,0.105954,-0.153572,-2.838582,3.747483,-1.125411,1.565741,-0.846813,1.140570
5,0.0,6th,0.008623,0.016851,-0.002447,-0.106568,0.080587,-0.145958,-1.916918,3.792987,-0.708752,1.566015,-0.565537,1.156530
6,0.0,7th,0.036522,0.040966,0.007449,-0.122248,0.081826,-0.098964,-1.034338,3.718784,-0.367491,1.527345,-0.299721,1.182961
7,0.0,8th,0.042200,0.118650,-0.026055,-0.115296,0.051019,-0.132149,-0.368778,3.775401,-0.135877,1.461038,-0.082316,1.199499


In [12]:
x = fft_left_hand.where(fft_left_hand.Time==0).dropna()
Amp_left_handX=np.array([])
Amp_left_handX = np.append(Amp_left_handX, np.sqrt(x.AccX_imag**2+x.AccX_real**2))
Amp_left_handX = Amp_left_handX/Amp_left_handX.sum()

In [13]:
y = fft_left_hand.where(fft_left_hand.Time==0).dropna()
Amp_left_handY=np.array([])
Amp_left_handY = np.append(Amp_left_handY, np.sqrt(x.AccY_imag**2+x.AccY_real**2))
Amp_left_handY = Amp_left_handY/Amp_left_handY.sum()

In [14]:
kld(Amp_left_handX,Amp_left_handY)

0.18769018026090117

In [15]:
Amp_left_handX

array([ 0.38004037,  0.14900622,  0.12697182,  0.07967777,  0.05248142,
        0.02007355,  0.05820118,  0.13354767])

In [16]:
fft_left_hand.Time.max()

6933

In [32]:
start_time = T.clock()
Amp_left_handAX=np.array([])
Amp_left_handAY=np.array([])
Amp_left_handAZ=np.array([])

Amp_left_handGX=np.array([])
Amp_left_handGY=np.array([])
Amp_left_handGZ=np.array([])

x = fft_right_leg.where(fft_left_hand.Time==0).dropna()

tmp =np.array( np.sqrt(x.AccX_imag**2+x.AccX_real**2) )
tmp =tmp/tmp.sum()
Amp_left_handAX = np.append(Amp_left_handAX, tmp)

tmp =np.array( np.sqrt(x.AccY_imag**2+x.AccY_real**2) )
tmp =tmp/tmp.sum()
Amp_left_handAY = np.append(Amp_left_handAY, tmp)

tmp =np.array( np.sqrt(x.AccZ_imag**2+x.AccZ_real**2) )
tmp =tmp/tmp.sum()
Amp_left_handAZ = np.append(Amp_left_handAZ, tmp)

for t in range(1,fft_right_leg.Time.max()):
    try:
        # 抽出したデータベースを格納
        x = fft_right_leg.where(fft_left_hand.Time==t).dropna()

        # 各軸、それぞれパワースペクトルを計算
        tmp =np.array( np.sqrt(x.AccX_imag**2+x.AccX_real**2) )
        tmp =tmp/tmp.sum()
        Amp_left_handAX = np.vstack((Amp_left_handAX, tmp))

        tmp =np.array( np.sqrt(x.AccY_imag**2+x.AccY_real**2) )
        tmp =tmp/tmp.sum()
        Amp_left_handAY = np.vstack((Amp_left_handAY, tmp))

        tmp =np.array( np.sqrt(x.AccZ_imag**2+x.AccZ_real**2) )
        tmp =tmp/tmp.sum()
        Amp_left_handAZ = np.vstack((Amp_left_handAZ, tmp))
    except ValueError:
        break
    
    
elapsed_time = T.clock() - start_time
print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"

elapsed_time:467.513038[sec]


In [18]:
start_time = T.clock()
Amp_right_legAX=np.array([])
Amp_right_legAY=np.array([])
Amp_right_legAZ=np.array([])

Amp_right_legGX=np.array([])
Amp_right_legGY=np.array([])
Amp_right_legGZ=np.array([])

x = fft_right_leg.where(fft_left_hand.Time==0).dropna()

tmp =np.array( np.sqrt(x.AccX_imag**2+x.AccX_real**2) )
tmp =tmp/tmp.sum()
Amp_right_legAX = np.append(Amp_right_legAX, tmp)

tmp =np.array( np.sqrt(x.AccY_imag**2+x.AccY_real**2) )
tmp =tmp/tmp.sum()
Amp_right_legAY = np.append(Amp_right_legAY, tmp)

tmp =np.array( np.sqrt(x.AccZ_imag**2+x.AccZ_real**2) )
tmp =tmp/tmp.sum()
Amp_right_legAZ = np.append(Amp_right_legAZ, tmp)

for t in range(1,fft_right_leg.Time.max()):
    try:
        # 抽出したデータベースを格納
        x = fft_right_leg.where(fft_left_hand.Time==t).dropna()

        # 各軸、それぞれパワースペクトルを計算
        tmp =np.array( np.sqrt(x.AccX_imag**2+x.AccX_real**2) )
        tmp =tmp/tmp.sum()
        Amp_right_legAX = np.vstack((Amp_right_legAX, tmp))

        tmp =np.array( np.sqrt(x.AccY_imag**2+x.AccY_real**2) )
        tmp =tmp/tmp.sum()
        Amp_right_legAY = np.vstack((Amp_right_legAY, tmp))

        tmp =np.array( np.sqrt(x.AccZ_imag**2+x.AccZ_real**2) )
        tmp =tmp/tmp.sum()
        Amp_right_legAZ = np.vstack((Amp_right_legAZ, tmp))
    except ValueError:
        break
    
    
elapsed_time = T.clock() - start_time
print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"

elapsed_time:462.906007[sec]


In [34]:
Amp_left_handAZ.shape

(6933, 8)

In [ ]:
start_time = T.clock()


kld_ll_rl_AX=np.array([])
kld_ll_rl_AY=np.array([])
kld_ll_rl_AZ=np.array([])
for x in Amp_left_handAX:
    for ax in Amp_right_legAX:
        kld_ll_rl_AX=np.append(kld_ll_rl_AX,kld(x,ax) )
    for ay in Amp_right_legAY:
        kld_ll_rl_AY=np.append(kld_ll_rl_AY,kld(x,ay) )
    for az in Amp_right_legAZ:
        kld_ll_rl_AZ=np.append(kld_ll_rl_AZ,kld(x,az) )

        
elapsed_time = T.clock() - start_time
print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"

In [43]:
kld_left_hand_AX[0]=np.append(kld_left_hand_AX[0],kld(x,y) )
kld_left_hand_AX[0]

array([], dtype=float64)

In [ ]:
start_time = T.clock()

elapsed_time = T.clock() - start_time
print ("elapsed_time:{0}".format(elapsed_time)) + "[sec]"